# Recommender Systems

### Learning Objectives:
- [Introduction: Simple Recommender Systems](#Introduction:-Simple-Recommeder-Systems)
- [Offline & Online Evaluation](#Offline-&-Online-Evaluation)
- [Content-based Recommenders](#Content\-based-Recommenders)
- [Collaborative-filtering](#Collaborative\-filtering)
- [Hybrid Systems](#Hybrid-Systems)


# Introduction: Simple Recommender Systems

__Recommender systems__, also referred to as __recommendation systems__, are filtering systems used by many different companies world-wide to be able to recommend products (e.g. movies, clothes, etc) based on user preferences. We obviously cannot recommend _exactly_ what a user wants as we cannot access or process all the information in their brain at the same time. Instead, we can take advantage or users'
past ratings and preferences to __predict__ the ratings of products users had not purchased/rated before and use these to estimate user preferences.

How do these systems do what they do? This is question that has become a large topic of research and the current answer is that there are mutliple ways to create recommender systems: each working under different assumptions and algorithms. There are two main broad classifications that we will cover shortly: __content-based recommendation__ and __collaborative filtering.__ 

Before we dive into these, we will cover what is informally referred to as a simple recommender system: a system that uses the weighted average rating from all users to make recommendations on the "best" options. Throughout this notebook, we will use the "Movies Dataset" from [Kaggle](https://www.kaggle.com/rounakbanik/the-movies-dataset), where the full version contains information on over 45,000 movies with 26 million ratings from  270,000 users. We will be using the small version, as shown below:

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
# Importing ratings
ratings = pd.read_csv("../DATA/ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
# Importing movie metadata
metadata = pd.read_csv("../DATA/movies_metadata.csv")
metadata.head()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


For our simple recommender system, we will use the IMDB's known __weighted average formula__ used for their Top Movies Chart, given as follows:

$$ R_{W} = (\frac{v}{v + m})R + (\frac{m}{v + m})C  $$

Where:
- $R_{W}$ is the weighted average movie rating
- $v$ is the number of votes for that movie title
- $m$ is the minimum number of votes required to be in the top Chart
- $R$ is the average rating of that movie title
- $C$ is the mean vote rating across all movies

We can now begin our calculations to construct our simple recommender:

In [4]:
# Computing mean vote count across all movies
vote_counts = metadata[metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = metadata[metadata['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

We must now choose a value for the minimum number. In this case, we will choose a value $m$ that gives us movies that have received more votes than 95% of the other remaining movies.

In [5]:
# Computing minimum number of votes required
m = vote_counts.quantile(0.95)
print(m)

434.0


We can now extract the movies that are considered to be canditates for the top charts in our recommender system given our computed 'm'.

In [6]:
# Extracting all movies that have a votecount that is greater than our m value
qualified = metadata[(metadata['vote_count'] >= m) & (metadata['vote_count'].notnull()) & (metadata['vote_average'].notnull())][['title', 'release_date', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [7]:
# Computing weighted average and determining top 250 chart
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['weighted_average'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('weighted_average', ascending=False).head(250)
qualified.head(15)

,title,release_date,vote_count,vote_average,popularity,genres,weighted_average
15480,Inception,2010-07-14,14075,8,29.1081,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",7.917588
12481,The Dark Knight,2008-07-16,12269,8,123.167,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",7.905871
22879,Interstellar,2014-11-05,11187,8,32.2135,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",7.897107
2843,Fight Club,1999-10-15,9678,8,63.8696,"[{'id': 18, 'name': 'Drama'}]",7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001-12-18,8892,8,32.0707,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.871787
292,Pulp Fiction,1994-09-10,8670,8,140.95,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",7.868660
314,The Shawshank Redemption,1994-09-23,8358,8,51.6454,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",7.864000
7000,The Lord of the Rings: The Return of the King,2003-12-01,8226,8,29.3244,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.861927
351,Forrest Gump,1994-07-06,8147,8,48.3072,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",7.860656
5814,The Lord of the Rings: The Two Towers,2002-12-18,7641,8,29.4235,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.851924


Great, so now we have the top charts. This chart can be carried further to become a simple recommender system by recommending the top movies in the charts to all users. Is this a good recommender system? Not particularly. By taking a global weighted average we are able to determine which ones are considered the best on average, but we are unable to account for the individual preferences of the users. For instance, if I was a fan of exclusively Romcoms, I would be recommended only movies I dislike from the list above. The same would happen to people who really don't like action and thrillers. Therefore, we must use our data to instead be able to account for individual preferences! Therefore, simple recommender systems like this that generate global recommendations are generally only used for users who the system has collected little data from.

By accounting for individual user preferences, we would likely achieve a higher score. But how can we determine which system is better? This leads us to the two methods of recommender system evaluation: __offline evaluation__ and __online evaluation.__

# Offline & Online Evaluation

How can we tell that our recommender system is doing what it is supposed to? There are two different approaches to evaluating our system:

- __Offline evaluation:__ Use data we already have and evaluation metrics to compute numeric efectiveness measures that can be tuned for and/or compared. These are the same evaluation metrics which we have encountered and used to assess the performance of our models
- __Online evaluation:__  involves using a live system, and tracking user-related behaviors such as dwell-times, click-through rates, and purchase conversions

When carrying offline evaluation, we can split our data into a training and a test dataset just as we have seen before to ensure that we are tuning our systems appropriately. On the other hand, online evaluation enables us to capture aspects of the performance of our system that offline methods cannot. Whether offline evaluation, online evaluation or a combination of both is the best method to evaluate our system's performance still remains a topic of research. For the purposes of this notebook, we will only be covering simplistic forms of offline evaluation.

# Content-based Recommenders
We can now begin to understand the first sub-class of recommendation systems: __content-based recommenders.__ Let us look at the recommendation problem in the context of our movies dataset. It is intuitive to say that we would like to recommend romance movies to someone that has rated other romantic movies highly as opposed to action, or to recommend older films to users to who are fans of old classics, or even Batman movies to a Batman fan. In this context, we are looking at the characteristics (content) of each movie, and recommending movies that are similar to the previously highly rated movies by the same user. 

There are multiple approaches for the machinery of content-based recommenders. Some will use the features of a movie you have just watched and recommend the most similar movies to that given movie given their respective features. However, we can take this a step further (given we have enough data) and use an algorithm to predict the best possible movie based on all the previously watched movies.

In modelling terms, we can frame the problem of recommendation as using the __features__ of movies watched by a user and the ratings given to each movie to __predict__ the rating the user would give to a movie not yet watched based on the movie's features. In other words, if we have enough data, we train a model for each user based on the previously watched movies and their features. The features we have chosen to use in our model are the degree of a given genre, release date and runtime. While release date and runtime are available, we have to extract and process the degree of given genres for each movie. Given that we have a list of genres for each movie, we will do the following:
- Determine how many genres there are and make each genre a feature of the model
- Assign to each genre a zero if it is not present in the movie's list of genres, or $\frac{1}{n}$ for each genre given a movie with $n$ genres

For example, with Toy Story, there are three genres: 'Animation', 'Comedy', and 'Family'. We will process this information by assigning $\frac{1}{3} = 0.33$ to each of these three categories and 0 to all others.

In [8]:
# Importing our data
links_small = pd.read_csv("../DATA/links_small.csv")
ratings_small = pd.read_csv("../DATA/ratings_small.csv")
md = pd.read_csv("../DATA/movies_metadata.csv")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
# Displaying our data
md
# ratings_small.head()
#links_small

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [10]:
# Making sure imdb_id matches between links and metadata
md["imdb_id"] = md["imdb_id"].str.strip("tt")

# Removing all movies without a genre
md["genres"].replace('[]', np.nan, inplace=True)
md.dropna(subset=["genres"], inplace=True)

# Removing all movies without available runtime
md.dropna(subset=["runtime"], inplace=True)

In [11]:
# Converting "genres" column from a dictionary from a list of strings, containing the respective genres
def extract_genres(x):
    genre_list = []
    x = eval(x) # executes expression inside of string
    for dictionary in x:
        genre_list.append(dictionary["name"])
    return genre_list
md["genres"] = md["genres"].apply(extract_genres)

Now that we have preprocessed the data we are going to use for this model, we can extract the three columns: genre, release_data and budget. We will now also determine the unique features present in the dataset and use each as a feature. Note that this model assumes that all possible genres are included in the dataset.

In [12]:
# Initialising our features matrix
FEATURES = md[["imdb_id", "release_date", "runtime"]]

# Finding unique genre names
GENRES = md["genres"]
unique_genres = list(set(GENRES.sum()))
print(unique_genres)

['Action', 'Horror', 'Western', 'Animation', 'Science Fiction', 'Thriller', 'Crime', 'Fantasy', 'Comedy', 'Romance', 'Foreign', 'Adventure', 'Drama', 'TV Movie', 'Documentary', 'Family', 'Music', 'History', 'Mystery', 'War']


In [15]:
# Adding each genre as a feature
data = []
for idx in range(len(GENRES)):
    row = np.zeros(len(unique_genres))
    genre_list = GENRES.iloc[idx]
    magnitude = (1/len(genre_list)) # magnitude given to each present genre
    print(genre_list, magnitude)
    for genre in genre_list:
        row[unique_genres.index(genre)] = magnitude
    data.append(row)
data = np.asarray(data)
        
# data = {genre:np.zeros(len(FEATURES["runtime"])) for genre in unique_genres}
extended_features = pd.DataFrame(data=data, columns=unique_genres)

# Horizontally stack our extended features and the original features
FEATURES = FEATURES.merge(extended_features, left_index=True, right_index=True)

['Animation', 'Comedy', 'Family'] 0.3333333333333333
['Adventure', 'Fantasy', 'Family'] 0.3333333333333333
['Romance', 'Comedy'] 0.5
['Comedy', 'Drama', 'Romance'] 0.3333333333333333
['Comedy'] 1.0
['Action', 'Crime', 'Drama', 'Thriller'] 0.25
['Comedy', 'Romance'] 0.5
['Action', 'Adventure', 'Drama', 'Family'] 0.25
['Action', 'Adventure', 'Thriller'] 0.3333333333333333
['Adventure', 'Action', 'Thriller'] 0.3333333333333333
['Comedy', 'Drama', 'Romance'] 0.3333333333333333
['Comedy', 'Horror'] 0.5
['Family', 'Animation', 'Adventure'] 0.3333333333333333
['History', 'Drama'] 0.5
['Action', 'Adventure'] 0.5
['Drama', 'Crime'] 0.5
['Drama', 'Romance'] 0.5
['Crime', 'Comedy'] 0.5
['Crime', 'Comedy', 'Adventure'] 0.3333333333333333
['Action', 'Comedy', 'Crime'] 0.3333333333333333
['Comedy', 'Thriller', 'Crime'] 0.3333333333333333
['Drama', 'Thriller'] 0.5
['Action', 'Adventure', 'Crime', 'Thriller'] 0.25
['Drama', 'Fantasy', 'Science Fiction', 'Thriller'] 0.25
['Drama', 'Romance'] 0.5
['Dram

['Drama'] 1.0
['Horror', 'Science Fiction', 'Thriller'] 0.3333333333333333
['Comedy', 'Crime', 'Drama', 'Music', 'Mystery', 'Romance'] 0.16666666666666666
['Comedy'] 1.0
['Drama', 'Romance'] 0.5
['Comedy', 'War'] 0.5
['Action', 'Drama', 'Thriller'] 0.3333333333333333
['Comedy', 'Romance'] 0.5
['Action', 'Adventure', 'Crime', 'Science Fiction', 'Thriller'] 0.2
['Comedy'] 1.0
['Action', 'Crime', 'Drama', 'Thriller'] 0.25
['Action', 'Drama', 'Thriller'] 0.3333333333333333
['Drama', 'Romance'] 0.5
['Drama'] 1.0
['Drama'] 1.0
['Drama'] 1.0
['Drama', 'History', 'War'] 0.3333333333333333
['Comedy', 'Drama'] 0.5
['Drama'] 1.0
['Drama'] 1.0
['Drama', 'Family', 'Fantasy'] 0.3333333333333333
['Comedy', 'Crime', 'Horror', 'Thriller'] 0.25
['Adventure', 'Fantasy', 'Action', 'Thriller', 'Science Fiction'] 0.2
['Drama', 'Romance'] 0.5
['Comedy', 'Drama'] 0.5
['Drama', 'Family'] 0.5
['Comedy', 'Drama'] 0.5
['Mystery', 'Drama', 'Comedy'] 0.3333333333333333
['Comedy', 'Drama', 'Romance'] 0.3333333333333

['Crime', 'Drama'] 0.5
['Documentary'] 1.0
['Comedy'] 1.0
['Comedy'] 1.0
['Crime', 'Drama'] 0.5
['Comedy', 'Horror'] 0.5
['Drama'] 1.0
['Drama'] 1.0
['Action', 'Adventure', 'Drama', 'Western'] 0.25
['Drama'] 1.0
['Drama'] 1.0
['Comedy'] 1.0
['Documentary'] 1.0
['Drama'] 1.0
['Mystery', 'Comedy', 'Crime'] 0.3333333333333333
['Comedy', 'Fantasy', 'Family'] 0.3333333333333333
['Adventure', 'Action', 'Thriller', 'Science Fiction'] 0.25
['Horror'] 1.0
['Science Fiction', 'Action'] 0.5
['Drama', 'Horror'] 0.5
['Drama'] 1.0
['Drama'] 1.0
['Drama'] 1.0
['Comedy'] 1.0
['Adventure', 'Comedy', 'Fantasy'] 0.3333333333333333
['Romance', 'Drama'] 0.5
['Drama'] 1.0
['Drama'] 1.0
['Comedy', 'Crime'] 0.5
['Documentary'] 1.0
['Documentary'] 1.0
['Documentary'] 1.0
['Animation', 'Comedy', 'Family'] 0.3333333333333333
['Documentary'] 1.0
['Mystery', 'Crime', 'Drama', 'History', 'Romance'] 0.2
['Animation', 'Drama', 'Fantasy'] 0.3333333333333333
['Drama', 'Thriller'] 0.5
['Crime'] 1.0
['Thriller', 'Crime']

['Science Fiction', 'Horror', 'Action'] 0.3333333333333333
['Horror', 'Science Fiction'] 0.5
['Drama', 'History', 'Mystery'] 0.3333333333333333
['Drama'] 1.0
['Drama', 'History'] 0.5
['Drama', 'History', 'Romance'] 0.3333333333333333
['Drama'] 1.0
['Comedy'] 1.0
['Comedy', 'Drama'] 0.5
['Comedy', 'Drama'] 0.5
['Comedy', 'Family', 'Science Fiction'] 0.3333333333333333
['Comedy'] 1.0
['Drama'] 1.0
['Comedy', 'Family'] 0.5
['Drama'] 1.0
['Horror', 'Drama'] 0.5
['Crime', 'Drama', 'Mystery', 'Thriller'] 0.25
['Comedy', 'Family'] 0.5
['Comedy', 'Horror', 'Thriller'] 0.3333333333333333
['Drama'] 1.0
['Horror', 'Mystery'] 0.5
['Thriller'] 1.0
['Drama'] 1.0
['Science Fiction'] 1.0
['Drama', 'Romance', 'Thriller'] 0.3333333333333333
['Adventure', 'Action', 'Thriller'] 0.3333333333333333
['Drama'] 1.0
['Romance', 'Drama'] 0.5
['Drama'] 1.0
['Drama', 'Adventure'] 0.5
['Drama', 'Romance'] 0.5
['Drama'] 1.0
['Comedy', 'Crime', 'Romance'] 0.3333333333333333
['Drama'] 1.0
['Comedy', 'Family'] 0.5
['Co

['Action', 'Adventure', 'Thriller'] 0.3333333333333333
['Romance', 'Drama'] 0.5
['Documentary'] 1.0
['Drama', 'Thriller', 'Crime', 'Romance'] 0.25
['Drama'] 1.0
['Drama', 'Crime', 'Thriller'] 0.3333333333333333
['Drama', 'War'] 0.5
['Crime', 'Drama', 'Thriller'] 0.3333333333333333
['Thriller', 'War'] 0.5
['Comedy', 'Romance'] 0.5
['Mystery', 'Thriller'] 0.5
['Adventure', 'Crime'] 0.5
['Mystery', 'Thriller'] 0.5
['Drama', 'Crime'] 0.5
['Thriller', 'Mystery'] 0.5
['Thriller'] 1.0
['Drama', 'Thriller'] 0.5
['Crime', 'Mystery', 'Thriller'] 0.3333333333333333
['Comedy', 'Drama', 'Thriller'] 0.3333333333333333
['Drama'] 1.0
['Music', 'Comedy'] 0.5
['Drama'] 1.0
['Mystery', 'Thriller'] 0.5
['Drama'] 1.0
['Drama', 'Thriller', 'Crime'] 0.3333333333333333
['Comedy'] 1.0
['Drama'] 1.0
['Drama'] 1.0
['Drama', 'Thriller'] 0.5
['Drama'] 1.0
['Crime', 'Drama', 'Thriller'] 0.3333333333333333
['Drama'] 1.0
['Drama', 'Crime'] 0.5
['Thriller', 'Science Fiction', 'Mystery'] 0.3333333333333333
['Drama'] 1.

...

['Drama'] 1.0
['Comedy', 'Drama', 'History', 'Romance'] 0.25
['Drama'] 1.0
['Drama', 'Thriller'] 0.5
['Documentary'] 1.0
['Documentary'] 1.0
['Documentary'] 1.0
['Documentary'] 1.0
['Thriller'] 1.0
['Documentary', 'Drama'] 0.5
['Drama'] 1.0
['Documentary'] 1.0
['Drama'] 1.0
['Documentary'] 1.0
['Thriller'] 1.0
['Thriller', 'Drama', 'Science Fiction'] 0.3333333333333333
['History', 'Drama', 'Action'] 0.3333333333333333
['Family', 'Fantasy', 'Adventure'] 0.3333333333333333
['Horror'] 1.0
['Horror', 'Comedy'] 0.5
['Horror'] 1.0
['Thriller'] 1.0
['Animation'] 1.0
['Drama'] 1.0
['Comedy', 'Drama'] 0.5
['Horror'] 1.0
['Drama'] 1.0
['Romance', 'Comedy'] 0.5
['Drama'] 1.0
['Drama'] 1.0
['Drama'] 1.0
['Drama'] 1.0
['Drama'] 1.0
['Drama'] 1.0
['Drama', 'Romance'] 0.5
['Drama'] 1.0
['Drama', 'Comedy'] 0.5
['Drama'] 1.0
['Drama', 'Romance'] 0.5
['Drama'] 1.0
['Thriller', 'Mystery'] 0.5
['Drama', 'Music', 'Romance'] 0.3333333333333333
['Drama'] 1.0
['Comedy', 'Drama'] 0.5
['Drama'] 1.0
['Animation'

['Romance', 'Drama', 'Comedy'] 0.3333333333333333
['Drama'] 1.0
['Documentary'] 1.0
['Drama'] 1.0
['Action', 'Adventure', 'Fantasy'] 0.3333333333333333
['Drama', 'Comedy'] 0.5
['Romance', 'Comedy', 'Family'] 0.3333333333333333
['Documentary'] 1.0
['Documentary', 'History'] 0.5
['Drama'] 1.0
['Drama', 'Comedy', 'Crime'] 0.3333333333333333
['Documentary'] 1.0
['Science Fiction', 'Comedy', 'Documentary'] 0.3333333333333333
['Drama', 'War', 'Thriller'] 0.3333333333333333
['Fantasy', 'Action', 'Adventure'] 0.3333333333333333
['Drama'] 1.0
['Documentary'] 1.0
['Romance', 'Drama'] 0.5
['Action', 'Crime', 'Foreign', 'Thriller'] 0.25
['Crime'] 1.0
['Documentary'] 1.0
['Comedy'] 1.0
['Science Fiction', 'Thriller'] 0.5
['Drama', 'History'] 0.5
['Crime', 'Drama'] 0.5
['Documentary'] 1.0
['Drama', 'Family', 'Romance'] 0.3333333333333333
['Drama', 'Action', 'Foreign'] 0.3333333333333333
['Drama'] 1.0
['Documentary'] 1.0
['Comedy'] 1.0
['Documentary'] 1.0
['Adventure', 'Action', 'Comedy'] 0.333333333

['Comedy'] 1.0
['Comedy'] 1.0
['Animation', 'Comedy', 'Adventure'] 0.3333333333333333
['Comedy'] 1.0
['Comedy', 'Action', 'Adventure'] 0.3333333333333333
['Horror'] 1.0
['Thriller', 'Horror'] 0.5
['Drama', 'Western'] 0.5
['Documentary'] 1.0
['Action', 'Crime'] 0.5
['Adventure', 'Drama', 'Fantasy', 'Science Fiction'] 0.25
['War', 'History'] 0.5
['Documentary'] 1.0
['Adventure', 'Comedy'] 0.5
['Drama', 'Romance'] 0.5
['Animation', 'Fantasy'] 0.5
['Comedy'] 1.0
['Documentary'] 1.0
['Thriller', 'Horror', 'Action'] 0.3333333333333333
['Drama'] 1.0
['Drama', 'Foreign', 'Romance'] 0.3333333333333333
['Drama', 'Documentary', 'Foreign'] 0.3333333333333333
['Drama'] 1.0
['Drama', 'War'] 0.5
['Horror', 'Comedy'] 0.5
['Horror'] 1.0
['Drama', 'Horror', 'Thriller'] 0.3333333333333333
['Action', 'Thriller'] 0.5
['Comedy', 'Horror'] 0.5
['Science Fiction', 'Action'] 0.5
['Comedy', 'Science Fiction', 'War'] 0.3333333333333333
['Comedy', 'Horror'] 0.5
['Horror', 'Mystery', 'Thriller'] 0.3333333333333333

['Drama', 'Fantasy', 'Horror', 'Science Fiction'] 0.25
['Family', 'Fantasy', 'Comedy', 'Romance'] 0.25
['Science Fiction', 'TV Movie', 'Action', 'Thriller'] 0.25
['Action', 'Adventure'] 0.5
['Action', 'Science Fiction'] 0.5
['Horror', 'Music'] 0.5
['Horror', 'Science Fiction'] 0.5
['Horror'] 1.0
['Horror'] 1.0
['Action', 'Science Fiction', 'Horror'] 0.3333333333333333
['Horror', 'Adventure', 'Fantasy'] 0.3333333333333333
['Action', 'Horror', 'Thriller'] 0.3333333333333333
['Adventure', 'Fantasy', 'Action'] 0.3333333333333333
['Action'] 1.0
['Fantasy', 'Horror', 'Science Fiction'] 0.3333333333333333
['Horror', 'Fantasy'] 0.5
['Horror', 'Science Fiction'] 0.5
['Fantasy', 'Drama'] 0.5
['Horror'] 1.0
['Horror', 'Science Fiction'] 0.5
['Science Fiction'] 1.0
['Thriller', 'Romance', 'Science Fiction'] 0.3333333333333333
['Drama'] 1.0
['TV Movie', 'Crime', 'Mystery', 'Thriller'] 0.25
['Drama', 'Science Fiction', 'War'] 0.3333333333333333
['Thriller'] 1.0
['Thriller'] 1.0
['Horror', 'Thriller'

['Comedy'] 1.0
['Drama'] 1.0
['Thriller'] 1.0
['Action', 'Science Fiction', 'Thriller', 'Adventure'] 0.25
['Drama'] 1.0
['Comedy'] 1.0
['Documentary'] 1.0
['Fantasy', 'Mystery', 'Thriller'] 0.3333333333333333
['Comedy'] 1.0
['Comedy', 'Drama'] 0.5
['Comedy'] 1.0
['Comedy'] 1.0
['Comedy'] 1.0
['Drama', 'Thriller'] 0.5
['Comedy'] 1.0
['Action', 'Crime', 'Adventure', 'Drama'] 0.25
['Comedy', 'Fantasy'] 0.5
['Comedy', 'Romance'] 0.5
['Animation', 'Comedy'] 0.5
['Animation'] 1.0
['Comedy', 'Romance'] 0.5
['Documentary'] 1.0
['Comedy'] 1.0
['Documentary'] 1.0
['Documentary'] 1.0
['Drama', 'Thriller', 'War'] 0.3333333333333333
['Adventure', 'Thriller'] 0.5
['Crime', 'Action', 'Thriller'] 0.3333333333333333
['Romance', 'Comedy', 'Crime'] 0.3333333333333333
['Romance', 'Comedy'] 0.5
['Documentary'] 1.0
['Drama', 'History', 'War'] 0.3333333333333333
['Crime'] 1.0
['Romance', 'Comedy'] 0.5
['Mystery', 'Comedy', 'Crime'] 0.3333333333333333
['Comedy'] 1.0
['Thriller', 'Horror', 'Mystery'] 0.3333333

In [16]:
FEATURES

,imdb_id,release_date,runtime,Action,Horror,Western,Animation,Science Fiction,Thriller,Crime,...,Foreign,Adventure,Drama,TV Movie,Documentary,Family,Music,History,Mystery,War
0,0114709,1995-10-30,81.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0
1,0113497,1995-12-15,104.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.333333,0.000000,0.0,0.0,0.333333,0.0,0.0,0.0,0.0
2,0113228,1995-12-22,101.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0114885,1995-12-22,127.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0113041,1995-02-10,106.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42834,1264961,2007-01-01,78.0,0.000000,0.0,0.0,0.000000,1.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
42835,0108346,1993-08-03,99.0,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
42836,0113075,1995-01-01,94.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.500000,0.0,0.0,0.500000,0.0,0.0,0.0,0.0
42837,1442486,2009-05-14,90.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


# Collaborative-filtering

# Hybrid Systems